#Reinforcement Learning Model

In [ ]:
import numpy as np 
import pandas as pd
from keras import layers
from keras import Input
from keras.models import Model
from keras.models import load_model
from keras import regularizers
from keras import optimizers
import random
import tensorflow as tf 
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import train_test_splits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from collections import deque
from tensorflow import keras
from tensorflow.keras import layers

#initialize random seeds so results are repeatable
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

DISCOUNT = 0.9
REPLAY_MEMORY_SIZE = 2000
MIN_REPLAY_MEMORY_SIZE = 512 #256
MIN_BATCH_SIZE = 128
EPISODES = 100
epsilon = 0.9 # 0.9
epsilon_decay = 0.99 #0.8
MIN_EPSILON = 0.0001
BATCH_SIZE=32
ALPHA=1
ALPHA_DECAY = 1 # 1 0.9 # 0.9999 #0.9975
ALPHA_MIN = 0.0001
ssc = StandardScaler()
ssc=MinMaxScaler()
digitizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')

# data = load_breast_cancer()
# print(data)
# train_data = data.data
# y = data.target
path = '/content/drive/MyDrive/Seizure detection CHB-MIT Dataset/'
iris = pd.read_csv(path+'sigmoid_fuc.csv')
train_data = np.array(iris.drop(columns = ['output']))
y = np.array(iris['output'])

labels = y

z = np.where(y == 1, 100,-100)
#oe = OneHotEncoder(handle_unknown='ignore', sparse=False)
#train_labels= oe.fit_transform(y.reshape(-1,1))
#digitizer followed by MinMaxScaler
train_data = digitizer.fit_transform(train_data)
ssc=MinMaxScaler()
train_data = ssc.fit_transform(train_data)


In [ ]:
class DQN():
    def __init__(self, input_size,output_size, data, labels):
        self.model = self.create_model(input_size, output_size)
        self.target_model = self.create_model(input_size, output_size)
        self.obs = data
        self.labels = labels
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        pass
    def create_model(self, input_size, output_size):
       
        input_tensor = Input(shape=(input_size,))
        x = layers.Dense(32,activation='relu')(input_tensor)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(32, activation='relu')(x)
        x = layers.Dense(16, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(16, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(8, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        output_tensor = layers.Dense(output_size, activation=None)(x)
        model = Model(inputs=input_tensor, outputs=output_tensor)
        #print (model.summary())
        ad = keras.optimizers.Adam(learning_rate=0.001) #'rmsprop'
        model.compile(optimizer=ad,loss='mse', metrics=['mae'])
        return model
    def play(self, obs, action):
        
        r = np.where((self.obs==obs).all(axis=1))
        if len (r[0]) == 0 :
            # something really wrong observation not in the game
            print("........... SOMETHING WRONG ................")
            return -100
        if self.labels[r[0][0]]  == action :
            return 100
        else :
            return -100
    def update_replay(self,obs,action,label, reward):
        self.replay_memory.append(((obs),action,label, reward))
    def get_qs(self, obs):
        return self.model.predict(obs, batch_size=1)
    def train(self):
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE :
            return
        minibatch = random.sample(self.replay_memory, MIN_BATCH_SIZE)
        # conevrt minibatch in list type to y numpy array. This is needed for access
        # also keras and TF except in numpy format
        y=np.array([np.array(xi, dtype=np.dtype(object)) for xi in minibatch])
        # extract features as a numpy array. As needed by Keras.
        # shape is (batch_size, feature_size)
        z=np.array([np.array(xi) for xi in y[:,0]])
        current_qa_list = dqn.model.predict(z, batch_size=BATCH_SIZE)
        future_qa_list = dqn.target_model.predict(z, batch_size=BATCH_SIZE)
        max_future_qa = np.argmax(future_qa_list, axis=1)
        alpha = ALPHA
        #qa = np.zeros(shape=(minibatch.shape[0],3))
        # Here only update the argmax because in theory we select max action
        # hence only the Q value of argmax get affected.
        for indx in range (y.shape[0]):
            #alpha1=1 gives 97 accuracy
            #current_qa_list[indx, y[indx, 1]] = (alpha1 * y[indx,3]) + ((1-alpha1)*max_future_qa[indx])
            #y[indx,1] is the action
            #y[indx,3] is the reward
            # z is the states / observation in numpy format
            #current_qa_list[indx, y[indx, 1]] = (alpha * y[indx,3]) + ((1-alpha)*max_future_qa[indx])
            current_qa_list[indx, y[indx, 1]] = current_qa_list[indx, y[indx, 1]] + (alpha * (y[indx,3])-current_qa_list[indx, y[indx, 1]] )
            pass
        dqn.model.fit(z, current_qa_list, epochs=16, batch_size=BATCH_SIZE, verbose=0)
        return 
    

#dqn = DQN(train_data.shape[1], 2, train_data, labels)
train_data, test_data, train_labels, test_labels = train_test_split(train_data, labels, test_size=0.3, random_state=True)
dqn = DQN(train_data.shape[1], 2, train_data, train_labels)
c=0
total_rewards = 0
while c < 1000:
    indx=np.random.randint(0,train_data.shape[0])
    act = train_labels[indx]
    act = dqn.get_qs(train_data[indx].reshape(1,-1))
    if np.random.rand() > epsilon:
        act = np.argmax(act)
    else:
        act = np.random.randint(0,high=2)
    r = dqn.play(train_data[indx], action=act)
    dqn.update_replay(train_data[indx],act, train_labels[indx], r)
    dqn.train()
    total_rewards = total_rewards + r

    c=c+1
    if ALPHA > ALPHA_MIN :
        ALPHA *= ALPHA_DECAY
    else :
        ALPHA = ALPHA_MIN
    
    if epsilon > MIN_EPSILON :
        epsilon *= epsilon_decay
    else :
        epsilon = MIN_EPSILON

    print("Iteration : ",c)
    print('ϵ : ',epsilon)
    print('reward : ',r)

    print("=================================================")

            
results = np.argmax(dqn.model.predict(train_data, batch_size=BATCH_SIZE), axis=1)
a= accuracy_score(train_labels,results) 
print(" train accuracy ", a)
tn, fp, fn, tp = confusion_matrix(train_labels, results).ravel()
print( " train tn ,", tn, " fp ", fp, " fn ", fn, " tp ", tp )
results = np.argmax(dqn.model.predict(test_data, batch_size=BATCH_SIZE), axis=1)
a= accuracy_score(test_labels,results) 
print(" Test accuracy ", a)
tn, fp, fn, tp = confusion_matrix(test_labels, results).ravel()
print( " test tn ,", tn, " fp ", fp, " fn ", fn, " tp ", tp )
#0.00001

Iteration :  1
ϵ :  0.891
reward :  -100
Iteration :  2
ϵ :  0.88209
reward :  -100
Iteration :  3
ϵ :  0.8732691
reward :  -100
Iteration :  4
ϵ :  0.8645364090000001
reward :  100
Iteration :  5
ϵ :  0.85589104491
reward :  100
Iteration :  6
ϵ :  0.8473321344609
reward :  -100
Iteration :  7
ϵ :  0.8388588131162911
reward :  100
Iteration :  8
ϵ :  0.8304702249851281
reward :  -100
Iteration :  9
ϵ :  0.8221655227352769
reward :  100
Iteration :  10
ϵ :  0.8139438675079241
reward :  100
Iteration :  11
ϵ :  0.8058044288328449
reward :  -100
Iteration :  12
ϵ :  0.7977463845445164
reward :  100
Iteration :  13
ϵ :  0.7897689206990712
reward :  100
Iteration :  14
ϵ :  0.7818712314920805
reward :  -100
Iteration :  15
ϵ :  0.7740525191771597
reward :  100
Iteration :  16
ϵ :  0.7663119939853881
reward :  100
Iteration :  17
ϵ :  0.7586488740455342
reward :  -100
Iteration :  18
ϵ :  0.7510623853050789
reward :  100
Iteration :  19
ϵ :  0.7435517614520281
reward :  100
Iteration :  20


In [ ]:
import numpy as np 
import pandas as pd
from keras import layers
from keras import Input
from keras.models import Model
from keras.models import load_model
from keras import regularizers
from keras import optimizers
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder
#from sklearn.model_selection import train_test_splits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer
from tensorflow import keras
from tensorflow.keras import layers

ssc = StandardScaler()
ssc=MinMaxScaler()
ssc = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')

# data = load_breast_cancer()
# train_data = data.data
# y = data.target
iris = pd.read_csv('/content/drive/MyDrive/Seizure detection CHB-MIT Dataset/sigmoid_fuc.csv')
train_data = np.array(iris.drop(columns = ['output']))
y = np.array(iris['output'])
print(train_data,y) 



oe = OneHotEncoder(handle_unknown='ignore', sparse=False)
train_labels= oe.fit_transform(y.reshape(-1,1))
train_labels = y
#digitizer followed by MinMaxScaler
train_data = ssc.fit_transform(train_data)
ssc=MinMaxScaler()
train_data = ssc.fit_transform(train_data)
s = [1,2,1,2]
a = np.array(s)
print(type(a))
class DQN():
    def __init__(self, input_size,output_size):
        self.model = self.create_model(input_size, output_size)
        self.target_model = self.create_model(input_size, output_size)
        pass
    def create_model(self, input_size, output_size):
       
        input_tensor = Input(shape=(input_size,))
        x = layers.Dense(32,activation='relu')(input_tensor)
        x = layers.Dropout(0.4)(x)
        x = layers.Dense(32, activation='relu')(x)
        x = layers.Dense(16, activation='relu')(x)
        x = layers.Dropout(0.4)(x)
        x = layers.Dense(16, activation='relu')(x)
        x = layers.Dropout(0.4)(x)
        x = layers.Dense(8, activation='relu')(x)
        x = layers.Dropout(0.4)(x)
        output_tensor = layers.Dense(output_size, activation='sigmoid')(x)
        model = Model(inputs=input_tensor, outputs=output_tensor)
        #print (model.summary())
        # rm = optimizers.RMSprop(learning_rate=0.001, momentum=1e-5) #'rmsprop'
        rm = keras.optimizers.Adam(learning_rate=0.001) #'rmsprop'
        model.compile(optimizer=rm,loss='binary_crossentropy', metrics=['acc'])
        return model


train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.3, random_state=True)
dqn = DQN(train_data.shape[1], 1)
for ep in range(128) :
    print('Episode : ',ep)
    dqn.model.fit(train_data, train_labels, epochs=1, batch_size=32)     
    e=dqn.model.evaluate(test_data, test_labels, batch_size=32)
    if (ep + 1) % 10 == 0 :
        dqn.target_model.set_weights(dqn.model.get_weights())
    #d = dqn.target_model.evaluate(test_data, test_labels, batch_size=32)
    print("=================================================================")

    #print(" e at ", ep, " ", e[1], " d ", d[1])
dqn.target_model.set_weights(dqn.model.get_weights())
e=dqn.model.evaluate(test_data, test_labels, batch_size=32)
d = dqn.target_model.evaluate(test_data, test_labels, batch_size=32)

print(' Final comparison ', e, " target = ", d)

results = dqn.model.predict(train_data, batch_size=32)
r = np.where( results > 0.5,1,0).reshape(-1,)
a= accuracy_score(train_labels, r )
print(" accuracy ", a)
tn, fp, fn, tp = confusion_matrix(train_labels, r).ravel()
print( " train tn ,", tn, " fp ", fp, " fn ", fn, " tp ", tp )
results = dqn.model.predict(test_data, batch_size=32)
r = np.where( results > 0.5,1,0).reshape(-1,)
a= accuracy_score(test_labels,r )
print(" test accuracy ", a)
tn, fp, fn, tp = confusion_matrix(test_labels, r).ravel()
print( " test tn ,", tn, " fp ", fp, " fn ", fn, " tp ", tp )


# dqn.model.save('c:/workdir/models')
# m2= load_model('c:/workdir/models')

# e = m2.evaluate(test_data, test_labels, batch_size=32)
# e=m2(test_data[0].reshape(1,-1), training=False)

# print(e.numpy())
# e=m2.predict(test_data[0].reshape(1,-1), batch_size=1)
# print(e)

[[-1.83209853]
 [-2.06056076]
 [-1.87572624]
 [-0.92395083]
 [-1.81445   ]
 [-2.02158286]
 [-1.37302377]
 [-0.57105162]
 [-0.37763121]
 [-0.30408269]
 [-0.16325779]
 [-2.31008639]
 [-2.11296624]
 [-2.29959103]
 [-1.69225842]
 [-1.75257942]
 [-3.12392409]
 [-2.20680355]
 [-2.84120884]
 [-1.59594448]
 [-2.2174532 ]
 [-2.28023009]
 [-2.59267059]
 [-2.66540892]
 [-2.14274696]
 [-2.53428558]
 [-2.78837777]
 [-1.50201174]
 [-1.95444637]
 [-1.29296297]
 [-1.23877142]
 [-2.68314618]
 [-1.64401315]
 [-2.44066514]
 [-1.67698555]
 [-1.55388247]
 [-3.02343406]
 [-2.28658216]
 [-2.28224907]
 [-2.59378927]
 [-2.00553489]
 [-2.31393858]
 [-2.06172024]
 [ 0.39116801]
 [-1.02226928]
 [-0.64588852]
 [-0.72056823]
 [-0.77910814]
 [-0.49552196]
 [-0.73018379]
 [-0.5816645 ]
 [-0.9273136 ]
 [-2.19431505]
 [-2.07940498]
 [-1.90174608]
 [-2.06678032]
 [-2.57453378]
 [-2.4492973 ]
 [-2.92636645]
 [-1.53057594]
 [-2.8340084 ]
 [-2.6985505 ]
 [-1.93009841]
 [-2.70319856]
 [-2.07173231]
 [-0.59305856]
 [-0.69102